In [1]:
import pandas as pd
import random, math

import numpy as np

from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [2]:
# 分類クラス
classes = ['chrysanthemum', 'cosmos', 'ginkgo', 'lotus' , 'margaret', 'rose']
nb_classes = len(classes)
batch_size = 32
nb_epoch = 1
current_dir = "/notebooks"
# image pixel
img_rows, img_cols = 224, 224

In [3]:
def build_model() :

    input_tensor = Input(shape=(img_rows, img_cols, 3))
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
    #vgg16.summary()

    _model = Sequential()
    
    _model.add(Flatten(input_shape=vgg16.output_shape[1:]))
    _model.add(Dense(256, activation='relu'))
    _model.add(Dropout(0.5))
    _model.add(Dense(nb_classes, activation='softmax'))

    model = Model(inputs=vgg16.input, outputs=_model(vgg16.output))
    # modelの14層目までのモデル重み
    for layer in model.layers[:15]:
        layer.trainable = False

    model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])
    return model

In [7]:
if __name__ == "__main__":
    print("train start")
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    train_generator = train_datagen.flow_from_directory(
        directory=current_dir + '/images',
        target_size=(img_rows, img_cols),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)

    test_datagen = ImageDataGenerator(rescale=1.0 / 255)
    
    test_generator = test_datagen.flow_from_directory(
        directory=current_dir + '/test_images',
        target_size=(img_rows, img_cols),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)
    
    model = build_model()    
    model.fit_generator(
        train_generator,
        steps_per_epoch=3000,
        epochs=nb_epoch,
        validation_data=test_generator,
        validation_steps=600
    )
        
    hdf5_file = current_dir + "/model/flower-model.hdf5"
    model.save_weights(hdf5_file)
    
    print("train finished")

Found 3000 images belonging to 6 classes.
Found 600 images belonging to 6 classes.
Epoch 1/1
3000/3000 [==============================] - 835s - loss: 0.4338 - acc: 0.8402 - val_loss: 0.3457 - val_acc: 0.8766
